In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import os

import pandas as pd
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v5 import CNN_v5

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v5((480, 640, 4), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v5/weights/"

def get_weights_paths(prefix):
    for directory, subdirectories, files in os.walk(prefix):
        yield from (os.path.join(directory, file) for file in files)

model_names = get_weights_paths(prefix)
model_names = list(model_names)

In [8]:
models = [CNN_v5((480, 640, 4), split_into_parts=20) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.7102 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 897ms/step - loss: 0.7167 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 840ms/step - loss: 0.7141 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.7133 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 839ms/step - loss: 0.7203 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 836ms/step - loss: 0.7153 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 875ms/step - loss: 0.7148 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.7116 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 853ms/step - loss: 0.7146 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 854ms/step - loss: 0.7123 - accuracy: 1.0000
Epoch 1/1
1/1 [========================

In [10]:
def as_rows(results):
    for name, cut_off_point_to_accuracy_with_difference in results.items():
        yield from ((f"{name}_{cut_off_point}", accuracy, counts - count_test) for cut_off_point, (accuracy, counts) in cut_off_point_to_accuracy_with_difference.items())

rows = as_rows(results)
rows = list(rows)

In [11]:
def calculate_mae_and_mse(rows):
    for *attributes, difference in rows:
        difference_mae = mae(difference)
        difference_mse = mse(difference)
        
        yield (*attributes, difference_mae, difference_mse)

mae_and_mse = calculate_mae_and_mse(rows)
mae_and_mse = list(mae_and_mse)

In [12]:
frame = pd.DataFrame(mae_and_mse, columns=["model", "accuracy", "mae", "mse"])
frame

,model,accuracy,mae,mse
0,CNN/CNN_v5/weights/cnn_v5_1e_6_0_0.1,0.000000,238.723333,57412.176667
1,CNN/CNN_v5/weights/cnn_v5_1e_6_0_0.2,0.000000,159.493333,25710.306667
2,CNN/CNN_v5/weights/cnn_v5_1e_6_0_0.30000000000...,0.000000,114.333333,13269.140000
3,CNN/CNN_v5/weights/cnn_v5_1e_6_0_0.4,0.000000,79.676667,6494.436667
4,CNN/CNN_v5/weights/cnn_v5_1e_6_0_0.5,0.275037,50.010000,2595.063333
...,...,...,...,...
130,CNN/CNN_v5/weights/cnn_v5_1e_6_14_0.5,79.874727,6.046667,56.200000
131,CNN/CNN_v5/weights/cnn_v5_1e_6_14_0.6000000000...,79.226772,6.523333,58.963333
132,CNN/CNN_v5/weights/cnn_v5_1e_6_14_0.7000000000...,50.996792,15.296667,260.956667
133,CNN/CNN_v5/weights/cnn_v5_1e_6_14_0.8,27.698580,22.556667,545.616667


In [13]:
frame.loc[frame.mae == frame.mae.min()]

,model,accuracy,mae,mse
95,CNN/CNN_v5/weights/cnn_v5_1e_6_10_0.6000000000...,87.932202,3.64,21.106667


In [14]:
frame.loc[frame.mse == frame.mse.min()]

,model,accuracy,mae,mse
95,CNN/CNN_v5/weights/cnn_v5_1e_6_10_0.6000000000...,87.932202,3.64,21.106667


In [15]:
frame.loc[frame.accuracy == frame.accuracy.max()]

,model,accuracy,mae,mse
95,CNN/CNN_v5/weights/cnn_v5_1e_6_10_0.6000000000...,87.932202,3.64,21.106667
